In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab_Notebooks/DL-Sigma2021/Deep Learning Project_SIGMA M2_2021"

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/DL-Sigma2021/Deep Learning Project_SIGMA M2_2021


In [6]:
import librosa
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import librosa.display
import scipy
import glob
import numpy as np
import math
import warnings
import pickle
from sklearn.utils import shuffle
import zipfile

In [2]:
windowLength = 255
overlap      = round(0.5 * windowLength) # overlap of 50%
ffTLength    = windowLength
inputFs      = 16e3
fs           = 16e3
numFeatures  = ffTLength//2 + 1
numSegments  = 16

In [19]:
def read_audio(filepath, sample_rate, normalize=True):
    """Read an audio file and return it as a numpy array"""
    audio, sr = librosa.load(filepath, sr=sample_rate)
    if normalize:
      div_fac = 1 / np.max(np.abs(audio)) / 3.0
      audio = audio * div_fac
    return audio, sr

def files_record(path,format_file='.WAV'):
    files = []
    name = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if format_file in file:
                files.append(os.path.join(r, file))
                name.append(file)
    return files

class FeatureExtractor:
    def __init__(self, audio, *, windowLength, overlap, sample_rate):
        self.audio = audio
        self.ffT_length = windowLength
        self.window_length = windowLength
        self.overlap = overlap
        self.sample_rate = sample_rate
        self.window = scipy.signal.hamming(self.window_length, sym=False)

    def get_stft_spectrogram(self):
        #return librosa.stft(self.audio, n_fft=self.ffT_length, win_length=self.window_length, hop_length=self.overlap,
                            #window=self.window, center=True)
        return scipy.signal.stft(self.audio, nfft=self.ffT_length, nperseg=self.window_length, noverlap=self.overlap,
                            window=self.window, padded=True)

def slice_stft(stft):
    mag = np.abs(stft[2])
    dt = mag.shape[1]
    if int(dt/numSegments)*numSegments!=dt:
        mag = mag[:,:int(dt/numSegments)*numSegments]
    split_stft = np.array_split(mag,int(mag.shape[1]/numSegments),axis=1)
    return split_stft
def segment_stack(stft):
    mag = np.abs(stft[2])
    sliced_image = slice_stft(stft)
    segments = np.zeros((mag.shape[0],numSegments,len(sliced_image)))
    for i in range(0,len(sliced_image)-1):
        stack = np.stack((sliced_image[i],sliced_image[i+1]),axis=-1)
        segments[:,:,i]= stack[:,:,0]
        if i+1 == len(sliced_image):
            segments[:,:,i+1]= stack[:,:,1]
    return segments

def built_dataset(filenames):
    #dataset = np.empty((m,numFeatures,1))
    for f in range(len(filenames)):
        Audio, sr = read_audio(os.path.join(filenames[f]), sample_rate=fs)
        AudioFeatureExtractor = FeatureExtractor(Audio, windowLength=windowLength, overlap=overlap, sample_rate=sr)
        stft_features = AudioFeatureExtractor.get_stft_spectrogram()
        seg = segment_stack(stft_features)
        if f == 0 :
            dataset = seg.copy()
        else:
            dataset = np.concatenate((dataset,seg),axis=2)
    return dataset

In [7]:
clean_path = "./dataset/speech_TRAIN_1/clean_TRAIN_1/"
noisy_path = './dataset/speech_TRAIN_1/noisy_TRAIN_1/'

In [20]:
# Clean Files Record (cfr)
# Noisy Files Record (nfr)
cfr = files_record(clean_path)
nfr = files_record(noisy_path)

In [22]:
train_dataset = built_dataset(nfr)
# np.save("train_1_v3",train_dataset)
train_dataset_out = built_dataset(cfr)
# np.save("train1_1_out_v3",train_dataset_out)

In [ ]:
noisy_path_val = './dataset/speech_TRAIN_2/noisy_TRAIN_2/'
clean_path_val = './dataset/speech_TRAIN_2/clean_TRAIN_2/'
noisy_path_test = './dataset/speech_TEST/noisy_TEST/'
clean_path_test = './dataset/speech_TEST/clean_TEST/'

In [ ]:
val_nfr = files_record(noisy_path_val)
val_cfr = files_record(clean_path_val)
test_nfr = files_record(noisy_path_test)
test_cfr = files_record(clean_path_test)

In [ ]:
validation_dataset_in = built_dataset(val_nfr)
# np.save("valdataset_in_v3",validation_dataset_in)
validation_dataset_out = built_dataset(val_cfr)
# np.save("valdataset_out_v3",validation_dataset_out)

test_dataset_in = built_dataset(test_nfr)
# np.save("testdataset_in_v3",test_dataset_in)
test_dataset_out = built_dataset(test_cfr)
# np.save("testdataset_out_v3",test_dataset_out)
